In [2]:
import numpy as np
import ripser

from scipy import sparse

In [1]:
def lower_star_img_new(D):
    """
    Construct a lower star filtration on an image. This function is essentially identical to the 
    lower_star_img() funciton from Ripser, except that it outputs H_1 persistence diagrams, in addition
    to H_0 diagrams.
    
    Parameters
    ----------
    D: ndarray (M, N)
        An array of image data
    Returns
    -------
    I: list
        A list containing the 0D and 1D persistence diagrams corresponding to the sublevelset filtration
    """
    idxs = np.arange(D.shape[0]*D.shape[1])
    idxs = np.reshape(idxs, D.shape)
    I = idxs.flatten()
    J = idxs.flatten()
    V = D.flatten()
    # Do 8 spatial neighbors
    tidxs = np.nan*np.ones((D.shape[0]+2, D.shape[1]+2), dtype=np.int64)
    tidxs[1:-1, 1:-1] = idxs
    tD = np.nan*np.ones_like(tidxs)
    tD[1:-1, 1:-1] = D
    for di in [-1, 0, 1]:
        for dj in [-1, 0, 1]:
            if di == 0 and dj == 0:
                continue
            thisJ = np.roll(tidxs, di, axis=0)
            thisJ = np.roll(thisJ, dj, axis=1)
            thisD = np.roll(tD, di, axis=0)
            thisD = np.roll(thisD, dj, axis=1)
            thisD = np.maximum(thisD, tD)
            # Deal with boundaries
            thisI = tidxs[np.isnan(thisD)==0]
            thisJ = thisJ[np.isnan(thisD)==0]
            thisD = thisD[np.isnan(thisD)==0]
            I = np.concatenate((I, thisI.flatten()))
            J = np.concatenate((J, thisJ.flatten()))
            V = np.concatenate((V, thisD.flatten()))
    sparseDM = sparse.coo_matrix((V, (I, J)), shape=(idxs.size, idxs.size))
    return ripser(sparseDM, distance_matrix=True, maxdim=1)['dgms']

In [2]:
def PC(dgm):
    beta0 = np.zeros(255)
    beta1 = np.zeros(255)
    
    dgm0 = dgm[0]
    dgm1 = dgm[1]
    for t in range(255):
        beta0[t] = np.sum((dgm0[:,0] < t)*(dgm0[:,1] > t))
        beta1[t] = np.sum((dgm1[:,0] < t)*(dgm1[:,1] > t))
        
    return beta0, beta1